In [ ]:
from eda_fn import*
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import recordlinkage as rl 
from recordlinkage.index import Block
import re

In [ ]:
df_frame = pd.read_csv(r"C:\Users\Invitado\Documents\Python\DS_MASTER\City_Adam_Intern\tbl_listing_info_amsterdam_20200104.csv") # Main data frame
df_new = pd.read_csv(r"C:\Users\Invitado\Documents\Python\DS_MASTER\City_Adam_Intern\tbl_listings_research_18_02_2022.csv") # Extra columns including more pictures 
df_extra = pd.read_csv(r"C:\Users\Invitado\Documents\Python\DS_MASTER\City_Adam_Intern\tbl_listing_description_07_03_2022.csv") # description of listing dutch/english

#Baseline data sets
df_phones_cat = pd.read_csv(r'C:\Users\Invitado\Documents\Python\DS_MASTER\City_Adam_Intern\phones_catalog.csv', sep='\|\|', engine='python')
df_phones_notcat = pd.read_csv(r'C:\Users\Invitado\Documents\Python\DS_MASTER\City_Adam_Intern\phones.csv', sep='\|\|', engine='python')
df_pairs_baseline = pd.read_csv(r"C:\Users\Invitado\Documents\Python\DS_MASTER\City_Adam_Intern\phones_phones_catalog_gold_standard_train.csv",sep=';')
df_phones_json_baseline_1 = pd.read_json(r'C:\Users\Invitado\Documents\Python\DS_MASTER\City_Adam_Intern\PhoneCatalog.json')
df_phones_json_baseline_2 = pd.read_json(r'C:\Users\Invitado\Documents\Python\DS_MASTER\City_Adam_Intern\phones.json')

In [ ]:
print('Main dataf has:',len(df_frame.columns))
print('New data has', len(df_new.columns))

In [ ]:
#df_frame.info()

In [ ]:
#df_new.info()

In [ ]:
#df_extra.info()

In [ ]:
print('total listings descriptions',len(set(list(df_extra['listingid']))))

In [ ]:
# From the frames this columns seem to contain the same entries Host_first_name== User_first_name == User_Smart_name == HOst_Smart_name 
# from df_frame custumer name == city == City in df_new
# Some of the columns of df_new refer to the same column in df_frame but with different name
df_frame.rename(columns={'customer_name':'City','house_name':'Name','bathrooms':'Bathrooms','bedrooms':'Bedrooms',
                        'beds':'Beds','listing_id':'ListingId','instant_bookable':'Instant_bookable','is_new_listing':'Is_new_listing',
                        'lat':'Lat','lng':'Lng','region':'Neighborhood','person_capacity':'Person_capacity','picture_url':'Picture_url',
                        'host_name':'Host_first_name','host_id':'Host_id','property_type':'Property_type','room_type_category':'Room_type_category',
                        'room_type':'Room_type','space_type':'Space_type','host_picture':'Host_picture_url'},inplace= True)

In [ ]:
histogram(df_frame,'Person_capacity')
histogram(df_frame,'Host_id')

In [ ]:
df_frame['ListingId'] = df_frame['ListingId'].str.replace('[A-Za-z]', '').astype('int64') # remove letters-ListingId has entries which start with a letter h123133... only in some sites
df_frame['Person_capacity'].fillna(0,inplace=True)   # There are Nan values which creates a conflict while converting all columns into the same dtype , turn them to 0 temporarily
df_frame['Host_id'].fillna(0,inplace=True) 
df_frame['Instant_bookable']= df_frame['Instant_bookable'].astype('int64') 
similar_columns = []
# Make every similar column match the dtype so it can be joined
for column in df_new.columns:
    if column in df_frame.columns: #Only use columns which are similar in both df
        dtype = df_new[column].dtypes
        dtype_2 = df_frame[column].dtypes
        similar_columns.append(column) # Save the similar keys in both dfs for joining
        if dtype == 'float64' and dtype!= dtype_2: # If the dtype needs to be float and they have different dtypes remove commas, numbers are 1,20 instead of 1.20 which is needed for conversion
            df_frame[column]= df_frame[column].str.replace(',', '.').astype(dtype)
        elif dtype!= dtype_2: # If they are not the same just change the dtype
            df_frame[column]= df_frame[column].astype(dtype) 




In [ ]:
histogram(df_frame,'Person_capacity')
histogram(df_frame,'Host_id')


In [ ]:
#Mergeing data 
df_merged = pd.merge(df_frame,df_new, on="ListingId",how='left', suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')

#Dataframes per site
df_bnb = df_merged[df_merged['site'] == 'AIRBNB']
df_book = df_merged[df_merged['site'] == 'Booking']
df_exp = df_merged[df_merged['site'] == 'Expedia']
df_trip = df_merged[df_merged['site'] == 'Tripadvisor']
df_verb = df_merged[df_merged['site'] == 'VRBO']

In [ ]:
print('AIRBNB total lisitngs : ',len(df_bnb))
print('Booking total lisitngs : ',len(df_book))
print('Expedia total lisitngs : ',len(df_exp))
print('Tripadvisor total lisitngs : ',len(df_trip))
print('VRBO total lisitngs : ',len(df_verb))

In [ ]:
labels = 'AIRBNB', 'Booking', 'Expedia', 'Tripadvisor','VRBO'
sizes = [21173, 621, 168, 523,688]


fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels, autopct='%.0f%%',startangle=0)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
ax1.legend()
plt.show()

In [ ]:
"""histogram(df_merged,'Bedrooms')
histogram(df_merged,'Bathrooms')
histogram(df_merged,'Beds')
histogram(df_merged,'bad_ratings_count')
histogram(df_merged,'Is_new_listing')
histogram(df_merged,'host_listings_count_hist')
histogram(df_merged,'kpi_capacity_more_than_4')
histogram(df_merged,'kpi_capacity_more_than_6')
histogram(df_merged,'2021-08_available_days')
histogram(df_merged,'host_reviews_per_listing')
histogram(df_merged,'host_popularity_score_percentage')
histogram(df_merged,'host_reviews_per_listing')
histogram(df_merged,'host_reviews_per_year')
histogram(df_merged,'Reviews_count')
histogram(df_merged,'Picture_count')"""
print(set(list(df_merged['site'])))

In [ ]:
#Checking complete postal code
complete_postalcode = 0
for x in df_merged['postalcode'].dropna():
    if re.search('[A-Za-z]', x):
        complete_postalcode+=1
print('Complete postal code format 1101MX :',complete_postalcode)
print('Total postalcode:',len(df_merged['postalcode'].dropna()))

In [ ]:
#Just information of the data sets do not tun
""" DROP:
    drop_list = ['Host_id','first_found_date','listing_published','listing_can_book','Listing_id_str',
                'Public_address','Localized_city',
                'User_Id','User_First_name','User_Smart_name','Host_smart_name','User_Thumbnail_url']
    
    Host_id : specific for each log in Airbnb only , for the rest 0 are Nan values therefore graph looks like it 
    first_found_date : complete only for bnb for the rest only 2 or 4 values are the same in all entries, why will this help the model
    listing_published: bnb an booking {-1,0} for the rest all {-1}
    listing_can_book: bnb{0,1} , for the rest 0 , what 
    Listing_id_str: another identifier only for bnb not extra info
    
    Host_thumbnail_url : same as Host profile pic but incomplete
    
    Public_address: bnb only seems similar to neighbourhood
    Localized_city: bnb only , same as neighbourhood just use neighbourhood as is more complete
    User_Id: bnb another identifier
    User_First_name: same as user smart name
    User_Smart_name: same as user smart name
    Host_smart_name: less entries than Host_first_name
    User_Thumbnail_url : same as host_Thumbnail_url
    **Xl_picture_url : seems to be same as picture url but only bnb double check if there are broken links 
    
    MAYBE DROP:
    
    Instant_bookable: Bookingtrip,exp,verb all 0s
    Is_new_listing: Bookingtrip,exp,verb all 0s
    
    Property_type_id: seems like categories, will it be useful ?
    Picture_count: How picture count will help ?
    *Is_business_travel_ready: bnb only {0,'False'} 
    *Is_family_preferred : bnb only {0,'False'}
    Host_has_profile_pic: only bnb -1,0,false,true will it be helpful?
    Host_is_superhost: only bnb -1,0,false,true will it be helpful?
    
    
    
    TO BE CLEANED:
    :) Name : Get rid of 'Aparthotel\r\nAmsterdam' that appears in Booking entries
           In Trip Advisor '\x91' \x92 appears quite often  
           Vrb \x96  w/
           Airbnb , expedia looks good
    :) Bedrooms: Clean for 0s many 0s means Nan?  Airbnb has 10,12,15,50
              exp seems good  
              Trip advisor only 0s
              Booking only 1
              vrbo seems complete
    :) Bathrooms: many 0s, in Booking, trip advisor, exp, vrbo all 0s actually
    
    :) Beds: trip, book, exp,verbo all 0s **ask if 0s are wrong for bnb checked multiple and actually wrong
    
    **Person_capacity: booking values seem weird {0, 5, 6, 7, 8, 12, 17} **checked and actually most 0 are wrong
                     trip advisor all 0s  
                     expedia, bnb, vrbo seems good  **cant access the websites for the rest the info seem to be wrong??
                     
    ?(maybe do not use) Property_type : bnb, verb okay , 
                    booking has but different categories , so how to combine?
                    trip,exp nan
                    
    :) Host_first_name : Only in Airbn and Vrbo complete for the rest are Nan values, 
                      book : 'unknown' , trip,exp: 'nan',
    
    
    
    
    kpi_capacity_more_than_6\4 : only bnb{-1,0}, rest nan                  
    :) kpi_instant_bookable: bnb complete, the rest have 0 values
    kpi_host_lists_10_or_more:  bnb complete, the rest nan
    2021-10_available_days: only bnb need to be converted to float {3,14}
    :)reviews_count : bnb,verb complete, book,exp empty, trip all 0.0
    
    ALMOST REDY TO USE:
    postalcode: complete for all however for bnb almost half of the entries after correcting for oukbend as nan
    Neighborhood: complete (maybe use for blooking)
    
    Lat,Lng : some nan but ready to use
    Picture_url: trip advisor, expedia empty the rest looks good (is link not pic already)
    listing_url : compelete 
    Host_picture_url:  bnb and vrbp only  ; Trip advisor, booking, expedia empty 
    
    kpi_host_lists_multiple: bnb only {-1,0}
    
    Host_thumbnail_url = only bnb picture of host small!
    
    Room_type_category: bnb only, the rest are nan
    Room_type: bnb only in ducth, the rest are nan
    Space_type: bnb only in ducth, the rest are nan
    booking_count: bnb complete rest nan
     
    host_listings_count_hist: bnb complete rest nan
    days_active: bnb complete rest nan
    host_start_month:  bnb complete rest nan
    host_start_year: bnb complete rest nan
    reviews_year,reviews_6\4\1_months,rating_avg,bad_ratings_count: bnb complete rest nan
"""

set(list(df_merged['site']))


In [ ]:
#drop Nan names? mostly incomplete check again after deciding which feautures to take 1285 Nan
# Columns which are mainly empty or have similar information that other columns 
drop = ['wsa_listing_id','city','City','2021-06_available_days','registration_number',
        '2021-07_available_days', '2022-01_available_days','Host_id','first_found_date','listing_published',
        'listing_can_book','Listing_id_str', 'Public_address','Localized_city',
        'User_Id','User_First_name','User_Smart_name','Host_smart_name','User_Thumbnail_url','Is_business_travel_ready','Is_family_preferred','User_Is_superhost','User_Has_profile_pic',
        'Property_type']
df_merged.drop(columns = drop ,inplace=True)

# Cleaning Postalcode
df_merged['postalcode'] = df_merged['postalcode'].map(lambda x: float("NaN") if x =='Onbekend' else x)

#Cleaning negatives ones , false and true entries
for column in df_merged.columns:
    try:
        df_merged[column] = df_merged[column].astype('int64')
    except ValueError:
        if column in  ['Host_is_superhost','Host_has_profile_pic']:
            df_merged[column] = df_merged[column].map(helper_negative_ones)
              
#Replace int by floats 198,00
week_days = ['2021-08_available_days',
       '2021-09_available_days', '2021-10_available_days',
       '2021-11_available_days', '2021-12_available_days',
       '2022-02_available_days', '2022-03_available_days',
       '2022-04_available_days']

for column in week_days:
       df_merged[column] = df_merged[column].str.replace(',', '.').astype('float64')


#Cleaning of data
df_merged['Name'] = df_merged['Name'].str.replace('Apartments\r\n', '').str.replace('Apartment\r\n', '').str.replace('Aparthotel\r\n', '')
df_merged['Name'] = df_merged['Name'].str.replace('\x92', '').str.replace('\x91', '').str.replace('\x96', '').str.replace('w/', '')
df_merged['Bedrooms'][df_merged['site']=='Booking'] = df_merged['Bedrooms'][df_merged['site']=='Booking'].replace(1.0,float("nan")) 
df_merged['Bedrooms'][df_merged['site']=='Tripadvisor'] = df_merged['Bedrooms'][df_merged['site']=='Tripadvisor'].replace(0,float("nan")) 
df_merged['Bathrooms'][df_merged['site']!='AIRBNB']= df_merged['Bathrooms'][df_merged['site']!='AIRBNB'].replace(0,float("nan"))
df_merged['Beds']= df_merged['Beds'].replace(0,float("nan"))
df_merged['Host_first_name'] = df_merged['Host_first_name'].replace('unknown', float("nan"))
df_merged['kpi_instant_bookable'][df_merged['site']!='AIRBNB'] =df_merged['kpi_instant_bookable'][df_merged['site']!='AIRBNB'].replace(0,float("nan"))
df_merged['reviews_count'][df_merged['site']=='Tripadvisor']= df_merged['reviews_count'][df_merged['site']=='Tripadvisor'].replace(0,float("nan"))
df_merged['days_active'] = df_merged['days_active'].str.replace(',', '.').astype('float64')
df_merged['reviews_year'] = df_merged['reviews_year'].str.replace(',', '.').astype('float64')
df_merged['reviews_6_months'] = df_merged['reviews_6_months'].str.replace(',', '.').astype('float64')
df_merged['reviews_3_months'] = df_merged['reviews_3_months'].str.replace(',', '.').astype('float64')
df_merged['reviews_1_month'] = df_merged['reviews_1_month'].str.replace(',', '.').astype('float64')
df_merged['bad_ratings_count'] = df_merged['bad_ratings_count'].str.replace(',', '.').astype('float64')
df_merged['year_total_available'] = df_merged['year_total_available'].str.replace(',', '.').astype('float64')

#df_merged['Property_type'] = df_merged['Property_type'].str.replace('\r\n1', '').str.replace('\r\n2', '').str.replace('\r\n4', '')

#Clean remove punctuation lowercase Neighborhood, string like values
df_merged['Neighborhood'] = df_merged['Neighborhood'].map(clean_string) 
df_merged['Name']= df_merged['Name'].map(clean_string)
df_merged['Room_type_category']= df_merged['Room_type_category'].map(clean_string)
df_merged['Room_type']= df_merged['Room_type'].map(clean_string)
df_merged['Space_type']= df_merged['Space_type'].map(clean_string) 
df_merged['Host_first_name']= df_merged['Host_first_name'].map(clean_string)

In [ ]:
# Cleaning and joining all descriptions for a listing
df_extra['listingid'] = df_extra['listingid'].astype(str)
df_extra = df_extra.groupby('listingid')['Description'].apply(join)
df_extra = pd.DataFrame(df_extra)
df_extra= df_extra.reset_index()
df_extra['listingid'] = df_extra['listingid'].astype(np.int64)

In [ ]:
df_extra.to_csv('descriptions.csv',index=False)

In [ ]:
df_extra_emb = pd.read_csv(r'C:\Users\Invitado\Documents\Python\DS_MASTER\City_Adam_Intern\df_extra_emb.csv')

In [ ]:
df_extra_emb.drop('Unnamed: 0',axis=1,inplace=True)
df_extra_emb

In [ ]:
df_merged = df_merged.merge(df_extra_emb,how='left',right_on='listingid',left_on='ListingId')
df_merged.drop('listingid',axis=1,inplace=True)

In [ ]:
df_red = df_merged[df_merged['last_found_date'].isna()]

In [ ]:
df_red.to_csv('df_red.csv',index=False)

In [ ]:
print('Total longitude',tot_nan_column(df_frame,'Lng'))
print('Total latitude',tot_nan_column(df_frame,'Lat'))
print('Total AirBnB longitude',tot_nan_column(df_bnb,'Lng'))
print('Total AirBnB latitude',tot_nan_column(df_bnb,'Lat'))
print('Total Booking longitude',tot_nan_column(df_book,'Lng'))
print('Total Booking latitude',tot_nan_column(df_book,'Lat'))
print('Total Expedia longitude',tot_nan_column(df_exp,'Lng'))
print('Total Expedia latitude',tot_nan_column(df_exp,'Lat'))
print('Total TripAD longitude',tot_nan_column(df_trip,'Lng'))
print('Total TripAD latitude',tot_nan_column(df_trip,'Lat'))
print('Total VRBO longitude',tot_nan_column(df_verb,'Lng'))
print('Total VRBO latitude',tot_nan_column(df_verb,'Lat'))

In [ ]:
print('Total non nan Lat',len(df_merged['Lat']))
print('Total non nan Lon',len(df_merged['Lng']))
print('Total non nan Lat non null',len(df_merged['Lat'].dropna()))
print('Total non nan Lon non null',len(df_merged['Lng'].dropna()))

In [ ]:
print('Total Postal codes',tot_nan_column(df_merged,'postalcode'),'out of', len(df_merged))
print('Total AirBnB Postal codes',tot_nan_column(df_bnb,'postalcode'),len(df_bnb))
print('Total Booking Postal codes',tot_nan_column(df_book,'postalcode'),len(df_book))
print('Total Expedia Postal codes',tot_nan_column(df_exp,'postalcode'),len(df_exp))
print('Total TripAD Postal codes',tot_nan_column(df_trip,'postalcode'),len(df_trip))
print('Total VRBO Postal codes',tot_nan_column(df_verb,'postalcode'),len(df_verb))

In [ ]:
df_merged.to_csv('merged_clean_df.csv')